In [21]:
# Download metadata to ./data/weather
import os
import urllib


# URLs for metadata
metadata_urls = {
    'station_inventory': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt', 
    'states': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-states.txt', 
    'stations': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt', 
    'countries': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-countries.txt'
}

# Output dir
directory = './data/weather'
if not os.path.exists(directory):
    os.makedirs(directory)

uo = urllib.URLopener()

for k, u in metadata_urls.items():
    uo.retrieve(u, './data/weather/'+k+'.txt')

In [1]:
# Download daily weather data summary by year
import requests
import gzip
import io
import csv

# URL for yearly data 
# Uncomment the yr_base_url and yrs to run the script

# yr_base_url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/'
# yrs = range(1997, 2015+1)

for yr in yrs:
    yr_url = ''.join([yr_base_url, str(yr), '.csv.gz'])
    
    print 'downloading file for: %d...' % (yr)
    resp = requests.get(yr_url, stream=True)
    csv_gz = resp.content
    f = io.BytesIO(csv_gz)
    print 'download complete'
    
    with gzip.GzipFile(fileobj=f, mode='rb') as fh, gzip.open('./data/weather/'+str(yr)+'.csv.gz', 'wb+') as fout:
        print 'filtering file...'
        for line in fh:
            if 'US' == line[:2]:
                fout.write(line)
        print 'finished writing file for: %d...\n' % (yr)

downloading file for: 2013...
download complete
filtering file...
finished writing file for: 2013...

downloading file for: 2014...
download complete
filtering file...
finished writing file for: 2014...

downloading file for: 2015...
download complete
filtering file...
finished writing file for: 2015...



In [13]:
# Filter US Stations to a separate file
stations_file = './data/weather/stations.txt'
us_stations = './data/weather/us_stations.txt'

with open(stations_file, 'rb') as fin, open(us_stations, 'wb+') as fout:
    for line in fin:
        if 'US' == line[:2]:
            fout.write(line)

# yrs = range(1997, 1997+1)
# print yrs


# for yr in yrs:
#     with gzip.open('./data/weather/'+str(yr)+'.csv.gz', mode='rb') as fh:
#         for line in fh:
#             print line


In [43]:
# Find the closest station for each fire 
import requests
import gzip
import io
import csv
import sys
from collections import defaultdict


wildfires = './data/wildfires_10k_sample.csv'
us_stations = './data/weather/us_stations.txt'

# load us stations latitude and longitude in a dictionary
us_latlon = defaultdict(str)
with open(us_stations, 'rb') as fus:
    for line in fus:
        station_code = line[:11].replace(' ', '')
        lat = line[11:20].replace(' ', '')
        lon = line[20:30].replace(' ', '')
        us_latlon[lat+','+lon] = station_code.upper()

# print us_latlon
hs = Haversine()

us_latlon_file = './data/weather/fire_weather_station.csv'
with open(wildfires, 'rb') as fwild, open(us_latlon_file, 'wb+') as fout:
    for line in fwild:
        line = line.strip().split(',')
        fire_code = line[0]
        fire_lat, fire_lon = map(float, [line[-2], line[-1]])
        
        closest_station = ['None', 0., 0., sys.maxsize, 0.]
        
        
        for key in us_latlon.keys():
            s_lat, s_lon = map(float, key.split(','))
            
            dist = hs.distance((fire_lat, fire_lon), (s_lat, s_lon))
            if dist <= closest_station[3]:
                closest_station[0] = us_latlon[key]
                closest_station[1] = s_lat
                closest_station[2] = s_lon                
                closest_station[3] = dist

        
        closest_station[4] = hs.bearing((fire_lat, fire_lon), closest_station[2])
        out_rec = [fire_code, fire_lat, fire_lon] + closest_station
        fout.write(','.join(map(str, out_rec))+'\n')

In [44]:
! head ./data/weather/fire_weather_station.csv

829373,30.7047,-89.3525,USR0000MHCK,30.6153,-89.4133,7.15834253377,144.335122146
1809170,38.6,-95.28,US1KSFR0013,38.6036,-95.2691,0.639144173568,139.377929358
431856,34.94978909,-84.35280409,USC00095917,34.9528,-84.3214,1.79109584946,144.9363249
710286,31.50096327,-89.10201294,USC00226626,31.4833,-89.0333,4.22948788372,144.174691615
880502,31.2072,-84.1956,USC00091500,31.1903,-84.2036,1.2601239291,146.289264691
1058606,35.25,-81.05,US1NCGS0009,35.249,-81.0335,0.933833718477,146.380419306
532620,35.13398,-82.70145,US1NCTR0024,35.1442,-82.7316,1.8445960404,145.62582336
675489,45.97181249,-91.35116212,USW00094973,46.0261,-91.4442,5.83349557081,136.995876083
709012,30.83872356,-89.09707676,US1MSST0005,30.856,-89.1249,2.03745403541,144.389308128
274347,45.97,-89.8877,USC00474320,45.95,-89.8833,1.39832841092,137.527232708


In [2]:
# Helper class for claculating distance between two lat, lon
# https://www.movable-type.co.uk/scripts/latlong.html

from math import *

class Haversine:
    def __init__(self, radius=3959.87433):
        self.RADIUS = radius
        
    def distance(self, src, dest):
        ''' Returns distance in miles
        '''
        lat1, lon1 = map(radians, src)
        lat2, lon2 = map(radians, dest)
        
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        
        return self.RADIUS * c
    
    def bearing(self, src, dest):
        '''Returns compass degrees
        '''
        lat1, lon1 = map(radians, src)
        lat2, lon2 = map(radians, dst)
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        y = sin(dlon) * cos(lon2)
        x = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
        bearing = atan2(y, x)
        bearing = degrees(bearing)
        return (bearing + 360) % 360
    
    
if __name__ == '__main__':
    dst = (-94.5930, -39.1230)
    src = (-94.4839, -39.1561)
    h = Haversine()
    print h.distance(src, dst)
    print h.bearing(src, dst)

7.54238357394
166.75532027


In [17]:
dst = (34.0, -118.0)
src = (34.0, -73.0)
h = Haversine()
print h.distance(src, dst)
print h.bearing(src, dst)

2556.79350884
67.7541634922
